# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1051 entries, 0 to 1050
Columns: 296 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(296)
memory usage: 2.4 MB


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

baseline_model.add(layers.Dense(units=100, activation='relu', input_shape=(n_features,))) 

baseline_model.add(layers.Dense(units=50, activation ='relu'))

# Output layer
baseline_model.add(layers.Dense(units=1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 131us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 52us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 41us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 45us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 41us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 43us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 41us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 38

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)


In [8]:

# Transform the validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [9]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [11]:
# Train the model 
history = normalized_input_model.fit(X_train,
                    y_train,
                    epochs=150,
                    batch_size=32,
                    validation_data=(X_val, y_val))



Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 147us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 54us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 44us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 41us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 39us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 41us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 42us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 41us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 40

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [12]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [13]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
history_normalized = normalized_model.fit(X_train,
                                          y_train_scaled,
                                          epochs=150,
                                          batch_size=32,
                                          validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 147us/step - loss: 0.4368 - mean_squared_error: 0.4368 - val_loss: 0.1992 - val_mean_squared_error: 0.1992
Epoch 2/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.2274 - mean_squared_error: 0.2274 - val_loss: 0.1613 - val_mean_squared_error: 0.1613
Epoch 3/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.1842 - mean_squared_error: 0.1842 - val_loss: 0.1588 - val_mean_squared_error: 0.1588
Epoch 4/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.1642 - mean_squared_error: 0.1642 - val_loss: 0.1376 - val_mean_squared_error: 0.1376
Epoch 5/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.1481 - mean_squared_error: 0.1481 - val_loss: 0.1321 - val_mean_squared_error: 0.1321
Epoch 6/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.1330 - mean_squared_error: 0.1330 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0317 - mean_squared_error: 0.0317 - val_loss: 0.1019 - val_mean_squared_error: 0.1019
Epoch 51/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0317 - mean_squared_error: 0.0317 - val_loss: 0.1032 - val_mean_squared_error: 0.1032
Epoch 52/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0309 - mean_squared_error: 0.0309 - val_loss: 0.1034 - val_mean_squared_error: 0.1034
Epoch 53/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0302 - mean_squared_error: 0.0302 - val_loss: 0.1034 - val_mean_squared_error: 0.1034
Epoch 54/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0292 - mean_squared_error: 0.0292 - val_loss: 0.1049 - val_mean_squared_error: 0.1049
Epoch 55/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0286 - mean_squared_error: 0.0286 - val_loss: 0.1042 - val_mean_squared_error: 0.104

1051/1051 [==============================] - 0s 41us/step - loss: 0.0143 - mean_squared_error: 0.0143 - val_loss: 0.1129 - val_mean_squared_error: 0.1129
Epoch 100/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1119 - val_mean_squared_error: 0.1119
Epoch 101/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1133 - val_mean_squared_error: 0.1133
Epoch 102/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0137 - mean_squared_error: 0.0137 - val_loss: 0.1126 - val_mean_squared_error: 0.1126
Epoch 103/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1130 - val_mean_squared_error: 0.1130
Epoch 104/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0133 - mean_squared_error: 0.0133 - val_loss: 0.1137 - val_mean_squared_error: 0.1137
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1181 - val_mean_squared_error: 0.1181
Epoch 149/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1178 - val_mean_squared_error: 0.1178
Epoch 150/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1190 - val_mean_squared_error: 0.1190


Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 24us/step


[0.007777665329691231, 0.007777665329691231]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [16]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 35us/step


[0.11902737866789669, 0.11902737866789669]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [17]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

27111.586925183463

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [22]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(units=100, activation='relu', input_shape=(n_features,), kernel_initializer='he_normal'))

# Add another hidden layer
             
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 439us/step - loss: 0.5083 - mean_squared_error: 0.5083 - val_loss: 0.1699 - val_mean_squared_error: 0.1699
Epoch 2/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.2217 - mean_squared_error: 0.2217 - val_loss: 0.1430 - val_mean_squared_error: 0.1430
Epoch 3/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.1797 - mean_squared_error: 0.1797 - val_loss: 0.1378 - val_mean_squared_error: 0.1378
Epoch 4/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.1569 - mean_squared_error: 0.1569 - val_loss: 0.1248 - val_mean_squared_error: 0.1248
Epoch 5/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.1411 - mean_squared_error: 0.1411 - val_loss: 0.1194 - val_mean_squared_error: 0.1194
Epoch 6/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1265 - mean_squared_error: 0.1265 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0303 - mean_squared_error: 0.0303 - val_loss: 0.1089 - val_mean_squared_error: 0.1089
Epoch 51/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0297 - mean_squared_error: 0.0297 - val_loss: 0.1090 - val_mean_squared_error: 0.1090
Epoch 52/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0293 - mean_squared_error: 0.0293 - val_loss: 0.1100 - val_mean_squared_error: 0.1100
Epoch 53/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0287 - mean_squared_error: 0.0287 - val_loss: 0.1102 - val_mean_squared_error: 0.1102
Epoch 54/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0281 - mean_squared_error: 0.0281 - val_loss: 0.1105 - val_mean_squared_error: 0.1105
Epoch 55/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0278 - mean_squared_error: 0.0278 - val_loss: 0.1100 - val_mean_squared_error: 0.110

1051/1051 [==============================] - 0s 40us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1194 - val_mean_squared_error: 0.1194
Epoch 100/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1192 - val_mean_squared_error: 0.1192
Epoch 101/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1192 - val_mean_squared_error: 0.1192
Epoch 102/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.1183 - val_mean_squared_error: 0.1183
Epoch 103/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0138 - mean_squared_error: 0.0138 - val_loss: 0.1185 - val_mean_squared_error: 0.1185
Epoch 104/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1208 - val_mean_squared_error: 0.1208
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1244 - val_mean_squared_error: 0.1244
Epoch 149/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1239 - val_mean_squared_error: 0.1239
Epoch 150/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1245 - val_mean_squared_error: 0.1245


Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 20us/step


[0.008181966803560333, 0.008181966803560333]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 34us/step


[0.12450862296293896, 0.12450862296293896]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [26]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(units=100, activation='relu', 
                          input_shape=(n_features,), 
                          kernel_initializer='lecun_normal'))


# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 203us/step - loss: 0.4769 - mean_squared_error: 0.4769 - val_loss: 0.1461 - val_mean_squared_error: 0.1461
Epoch 2/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.2096 - mean_squared_error: 0.2096 - val_loss: 0.1230 - val_mean_squared_error: 0.1230
Epoch 3/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.1698 - mean_squared_error: 0.1698 - val_loss: 0.1168 - val_mean_squared_error: 0.1168
Epoch 4/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.1479 - mean_squared_error: 0.1479 - val_loss: 0.1085 - val_mean_squared_error: 0.1085
Epoch 5/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.1330 - mean_squared_error: 0.1330 - val_loss: 0.1060 - val_mean_squared_error: 0.1060
Epoch 6/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.1195 - mean_squared_error: 0.1195 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0267 - mean_squared_error: 0.0267 - val_loss: 0.1161 - val_mean_squared_error: 0.1161
Epoch 51/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0261 - mean_squared_error: 0.0261 - val_loss: 0.1171 - val_mean_squared_error: 0.1171
Epoch 52/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0259 - mean_squared_error: 0.0259 - val_loss: 0.1186 - val_mean_squared_error: 0.1186
Epoch 53/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0253 - mean_squared_error: 0.0253 - val_loss: 0.1191 - val_mean_squared_error: 0.1191
Epoch 54/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0247 - mean_squared_error: 0.0247 - val_loss: 0.1188 - val_mean_squared_error: 0.1188
Epoch 55/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0244 - mean_squared_error: 0.0244 - val_loss: 0.1187 - val_mean_squared_error: 0.118

1051/1051 [==============================] - 0s 41us/step - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.1290 - val_mean_squared_error: 0.1290
Epoch 100/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1278 - val_mean_squared_error: 0.1278
Epoch 101/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1285 - val_mean_squared_error: 0.1285
Epoch 102/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0123 - mean_squared_error: 0.0123 - val_loss: 0.1268 - val_mean_squared_error: 0.1268
Epoch 103/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0120 - mean_squared_error: 0.0120 - val_loss: 0.1276 - val_mean_squared_error: 0.1276
Epoch 104/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0119 - mean_squared_error: 0.0119 - val_loss: 0.1293 - val_mean_squared_error: 0.1293
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1348 - val_mean_squared_error: 0.1348
Epoch 149/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1332 - val_mean_squared_error: 0.1332
Epoch 150/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1344 - val_mean_squared_error: 0.1344


Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 31us/step


[0.007015991652496053, 0.007015991652496053]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [28]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 34us/step


[0.1344344986907203, 0.1344344986907203]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [31]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mse'])


# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 250us/step - loss: 0.3205 - mean_squared_error: 0.3205 - val_loss: 0.1370 - val_mean_squared_error: 0.1370
Epoch 2/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.1666 - mean_squared_error: 0.1666 - val_loss: 0.1106 - val_mean_squared_error: 0.1106
Epoch 3/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.1243 - mean_squared_error: 0.1243 - val_loss: 0.1228 - val_mean_squared_error: 0.1228
Epoch 4/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0989 - mean_squared_error: 0.0989 - val_loss: 0.1103 - val_mean_squared_error: 0.1103
Epoch 5/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0889 - mean_squared_error: 0.0889 - val_loss: 0.0988 - val_mean_squared_error: 0.0988
Epoch 6/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0664 - mean_squared_error: 0.0664 - val_lo

1051/1051 [==============================] - 0s 42us/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.1112 - val_mean_squared_error: 0.1112
Epoch 50/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0104 - mean_squared_error: 0.0104 - val_loss: 0.1143 - val_mean_squared_error: 0.1143
Epoch 51/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0071 - mean_squared_error: 0.0071 - val_loss: 0.0918 - val_mean_squared_error: 0.0918
Epoch 52/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0923 - val_mean_squared_error: 0.0923
Epoch 53/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0087 - mean_squared_error: 0.0087 - val_loss: 0.0942 - val_mean_squared_error: 0.0942
Epoch 54/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0074 - mean_squared_error: 0.0074 - val_loss: 0.0893 - val_mean_squared_error: 0.0893
Epoch 55/15

1051/1051 [==============================] - 0s 43us/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0916 - val_mean_squared_error: 0.0916
Epoch 99/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0046 - mean_squared_error: 0.0046 - val_loss: 0.0908 - val_mean_squared_error: 0.0908
Epoch 100/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0055 - mean_squared_error: 0.0055 - val_loss: 0.0870 - val_mean_squared_error: 0.0870
Epoch 101/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0946 - val_mean_squared_error: 0.0946
Epoch 102/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0046 - mean_squared_error: 0.0046 - val_loss: 0.0928 - val_mean_squared_error: 0.0928
Epoch 103/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0934 - val_mean_squared_error: 0.0934
Epoch 1

Epoch 147/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0845 - val_mean_squared_error: 0.0845
Epoch 148/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0893 - val_mean_squared_error: 0.0893
Epoch 149/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0781 - val_mean_squared_error: 0.0781
Epoch 150/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 0.0849 - val_mean_squared_error: 0.0849


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [32]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 32us/step


[0.014318229104228296, 0.014318229104228296]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 34us/step


[0.08489244655057958, 0.08489244655057958]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [34]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam',
              loss='mse',
              metrics=['mse'])


# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 282us/step - loss: 0.3809 - mean_squared_error: 0.3809 - val_loss: 0.1649 - val_mean_squared_error: 0.1649
Epoch 2/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.1611 - mean_squared_error: 0.1611 - val_loss: 0.1321 - val_mean_squared_error: 0.1321
Epoch 3/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.1128 - mean_squared_error: 0.1128 - val_loss: 0.1039 - val_mean_squared_error: 0.1039
Epoch 4/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0862 - mean_squared_error: 0.0862 - val_loss: 0.1015 - val_mean_squared_error: 0.1015
Epoch 5/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0740 - mean_squared_error: 0.0740 - val_loss: 0.1003 - val_mean_squared_error: 0.1003
Epoch 6/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0562 - mean_squared_error: 0.0562 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.1044 - val_mean_squared_error: 0.1044
Epoch 51/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0968 - val_mean_squared_error: 0.0968
Epoch 52/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.1046 - val_mean_squared_error: 0.1046
Epoch 53/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0028 - mean_squared_error: 0.0028 - val_loss: 0.1023 - val_mean_squared_error: 0.1023
Epoch 54/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0027 - mean_squared_error: 0.0027 - val_loss: 0.1020 - val_mean_squared_error: 0.1020
Epoch 55/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.0973 - val_mean_squared_error: 0.097

Epoch 99/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0947 - val_mean_squared_error: 0.0947
Epoch 100/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0930 - val_mean_squared_error: 0.0930
Epoch 101/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0941 - val_mean_squared_error: 0.0941
Epoch 102/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0916 - val_mean_squared_error: 0.0916
Epoch 103/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0928 - val_mean_squared_error: 0.0928
Epoch 104/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0912 - val_mean_squared_error: 

1051/1051 [==============================] - 0s 46us/step - loss: 7.5816e-04 - mean_squared_error: 7.5816e-04 - val_loss: 0.0820 - val_mean_squared_error: 0.0820
Epoch 148/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0886 - val_mean_squared_error: 0.0886
Epoch 149/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0820 - val_mean_squared_error: 0.0820
Epoch 150/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0909 - val_mean_squared_error: 0.0909


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [35]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 40us/step


[0.0021368650435740315, 0.0021368650435740315]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [36]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 37us/step


[0.09087603805522955, 0.09087603805522955]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [38]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 47us/step


[0.14103698975419346, 0.14103698975419346]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [43]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
RMSE = np.sqrt(mean_squared_error(y_test, y_test_pred))
RMSE

29511.95775722832

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.